### 1. Import the essential libraries

In [1]:
import os
import sys
import cv2
import random
import joblib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

from skimage.color import rgb2lab, lab2rgb
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

from tqdm import tqdm_notebook

### 1.1 Define some helpful functions

In [2]:
def surf_feature_extraction_v2(image, indices, hessianThreshold=400, nOctaves=3, n_keypoints=10, surf_window=20):
    # construct a mask for the segment
    surf = cv2.xfeatures2d.SURF_create(hessianThreshold=hessianThreshold, nOctaves=nOctaves)
    key_points = [cv2.KeyPoint(y, x, surf_window) for (x,y) in indices]
    key_points = sorted(key_points, key=lambda x: -x.response)[:n_keypoints]
    key_points, description = surf.compute(image, key_points)
    
    if len(key_points) != 0:
        description = description.flatten()
    else:
        description = np.array([])
        
    fearute_vector_size = (n_keypoints * 64)
    if description.size < fearute_vector_size:
        description = np.concatenate([description, np.zeros(fearute_vector_size - description.size)])
    
    return (key_points, description)

In [3]:
def calc_energy_and_amplitude(filtered_img):
    '''
    Return a list of values:
      * Local Energy = summing up the squared value of each matrix value from a response matrix
      * Mean Amplitude = sum of absolute values of each matrix value from a response matrix
    '''
    local_energy = np.sum(np.square(filtered_img))
    mean_amplitude = np.sum(np.absolute(filtered_img))
    return [local_energy, mean_amplitude]

def gabor_feature_extraction_v2(image_lab_L, theta_range=[0, np.pi/6, np.pi/4, np.pi/3, np.pi/2, 2*np.pi/3, 3*np.pi/4, 5*np.pi/6], scale_range=[3, 6, 13, 28, 58]):
    sp_local_energy = []
    sp_mean_amplitude = []
    for scale in scale_range:
        for angle in theta_range:
            g_kernel = cv2.getGaborKernel(ksize=(20, 20), sigma=scale, theta=angle, lambd=7, gamma=0.9, psi=1.5, ktype=cv2.CV_32F)
            filtered_img = cv2.filter2D(image_lab_L, cv2.CV_8UC3, g_kernel)
            
            lcl_enrgy, mn_amplitude = calc_energy_and_amplitude(filtered_img)
            
            sp_local_energy.extend([lcl_enrgy])
            sp_mean_amplitude.extend([mn_amplitude])
            
    return (sp_local_energy, sp_mean_amplitude)

### 2. Set the paths for *train* and *test* data

In [4]:
train_images_path = os.path.join('.', 'data', 'Train')
res_path          = os.path.join('.', 'res_all')

### 2.1. Get the Train Images & the Color Palette

In [5]:
train_images = joblib.load(os.path.join(res_path,'trainImages.joblib'))
colorPalette = joblib.load(os.path.join(res_path,'colorPalette.joblib'))
print (f'{type(train_images)}\n{type(colorPalette)}')

<class 'numpy.ndarray'>
<class 'sklearn.cluster.k_means_.MiniBatchKMeans'>


In [6]:
np.sort(train_images)

array(['0024.png', '0090.png', '0103.png', '0142.png', '0191.png',
       '0205.png'], dtype='<U8')

### 3. Get Images and Perform the Preprocessing (cont.)
  * #### 3.1: Read Image
  * #### 3.2: Normalize Values from [0, 255] to [0, 1]
  * #### 3.3: Convert Image from RGB to LAB
  * #### 3.4: Get the Superpixels for an Image using the SLIC Segmentation Algorithm
  * #### 3.5: Extract the SURF Features from an Image per Superpixel
  * #### 3.6: Extract the Gabor Features from an Image per Superpixel
  * #### 3.7: Get the Dominant Color per Superpixel
  * #### 3.8: Associate each feature vector {SURF, Gabor} with the corresponding color class of the dominant color in the color palette

In [11]:
dataset = None

for img in tqdm_notebook(train_images):
    # 3.1: Read Image
    trainImage = cv2.imread(os.path.join(train_images_path, img))
    trainImage = cv2.cvtColor(trainImage, cv2.COLOR_BGR2RGB)
    # 3.2: Normalize Values from [0, 255] to [0, 1]
    trainImageScaled = trainImage/255
    # 3.3: Convert Image from RGB to LAB
    trainImageLAB = rgb2lab(trainImageScaled)
    # 3.4: Get the Superpixels for an Image using the SLIC Segmentation Algorithm
    segments = slic(trainImageLAB, convert2lab=False, n_segments=600, sigma=5)
    # 3.5: Extract the SURF Features from an Image per Superpixel
    # 3.6: Extract the Gabor Features from an Image per Superpixel
    # 3.7: Get the Dominant Color per Superpixel
    for (i, segment) in enumerate(np.unique(segments)):
#         print (f'Processed superpixel: {segment}\t Color Class: {color_vector}\r', end='')Z
        # Construct a mask for the segment
        mask = np.zeros(trainImage.shape[:2], dtype = "uint8")
        mask[segments == segment] = 255
        superpixel = cv2.bitwise_and(trainImageLAB, trainImageLAB, mask=mask)
                
        superpixel_colors = pd.DataFrame(np.vstack(superpixel[segments==segment, 1:3]), columns=['a', 'b'])    
#         superpixel_colors = pd.DataFrame(np.vstack(superpixel[:, :, 1:3]), columns=['a', 'b'])    

        maximal_color = KMeans(n_clusters=5, n_init=4, max_iter=100, n_jobs=-1)
        maximal_color.fit(superpixel_colors.values)
        superpixel_colors['y_color'] = maximal_color.labels_
        superpixel_dominant_color = superpixel_colors.loc[superpixel_colors['y_color'] == superpixel_colors['y_color']  
                                                     .value_counts().index[0]].apply(lambda x: x.median())[['a', 'b']]  
#         superpixel_dominant_color = superpixel_colors.loc[superpixel_colors['y_color'] == superpixel_colors['y_color']  
#                                                      .value_counts().index[1]].apply(lambda x: x.median())[['a', 'b']]  

        superpixel_dominant_color_class = colorPalette.predict(superpixel_dominant_color.values.reshape(1,-1))
          
        surf_vector = surf_feature_extraction_v2(np.uint8(trainImageLAB[:,:,0]), np.argwhere(segments == segment), n_keypoints=7, surf_window=20)[1] 
        gabor_vector = np.hstack(gabor_feature_extraction_v2(np.uint8(superpixel[:,:,0])))
        color_vector = np.array(superpixel_dominant_color_class[0])
        feature_vector = np.hstack((surf_vector, gabor_vector, color_vector))
        
        if (dataset is None):
            dataset = np.array(feature_vector)
        else:
            dataset = np.row_stack((dataset, feature_vector))

/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_y = grid_y[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:158: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklear

/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  segments_z = grid_z[slices]
/home/andretri7/anaconda3/lib/python3.6/site-packages/skimage/segmentation/slic_superpixels.py:157: FutureWarning: Using a non-t

In [12]:
pdDataset = pd.DataFrame(dataset)
pdDataset

,0,1,2,3,4,5,6,7,8,9,...,519,520,521,522,523,524,525,526,527,528
0,-0.001489,-0.000555,0.012909,0.036649,-0.060788,-0.026786,0.084633,0.152681,0.026609,0.027223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.002336,0.004140,0.006236,0.015309,-0.016316,0.148843,0.100046,0.173304,-0.002657,0.019892,...,285.0,264.0,302.0,422.0,541.0,308.0,471.0,376.0,291.0,0.0
2,-0.002494,-0.012410,0.002771,0.014007,-0.020176,-0.047946,0.023563,0.051405,0.013172,-0.004436,...,404.0,774.0,382.0,278.0,272.0,81.0,344.0,350.0,422.0,0.0
3,-0.000938,-0.001349,0.005529,0.012502,-0.060632,-0.047526,0.072900,0.086925,0.012383,-0.007179,...,1341.0,2703.0,1645.0,1273.0,1065.0,434.0,935.0,1033.0,1398.0,0.0
4,-0.001241,0.000811,0.001241,0.000811,0.014634,0.022312,0.014634,0.022312,0.000466,0.008351,...,2049.0,2760.0,2121.0,1835.0,1514.0,566.0,1511.0,1838.0,2120.0,0.0
5,-0.000728,0.000829,0.000728,0.000829,0.004545,0.004184,0.023788,0.020895,0.015311,0.021063,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.001073,-0.000453,0.002378,0.001045,0.017127,0.013522,0.024944,0.030162,-0.012071,-0.012377,...,2910.0,4032.0,2795.0,2435.0,1987.0,511.0,2166.0,2643.0,3007.0,0.0
7,-0.001829,0.001697,0.002341,0.002186,0.008742,0.014413,0.039878,0.074249,-0.017476,-0.039416,...,3273.0,4555.0,3640.0,3592.0,3343.0,1324.0,3183.0,3384.0,3394.0,0.0
8,-0.042031,0.035632,0.042031,0.046175,-0.006300,-0.013533,0.016525,0.018000,0.022418,-0.021893,...,467.0,462.0,484.0,635.0,768.0,547.0,767.0,619.0,482.0,0.0
9,-0.002297,0.002593,0.006314,0.003634,-0.014239,0.014869,0.033781,0.023660,0.001114,0.001037,...,134.0,132.0,135.0,156.0,154.0,104.0,154.0,156.0,135.0,0.0


# CHECKPOINT
* ### Save the train image dataset for the following notebooks

In [13]:
if not os.path.exists(os.path.join('.', 'res_all')):
    os.makedirs(os.path.join('.', 'res_all'))
    pdDataset.to_pickle(os.path.join(save_path, 'pdDataset.pkl'))
save_path = os.path.join('.', 'res_all')

In [14]:
pdDataset.to_pickle(os.path.join(save_path, 'pdDataset.pkl'))